# Методы сбора и обработки данных из сети Интернет
## GeekBrains
## Практическая работа
## Казанцев Виталий

### Задача
Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
*        название источника;
*        наименование новости;
*        ссылку на новость;
*        дата публикации.
 
     **Сложить собранные новости в БД**

Минимум один сайт, максимум - все три


Подключаем нужные библиотеки

In [1]:
from lxml import html
from pprint import pprint
from pymongo import MongoClient as mcl
import requests

In [2]:
class NewsScraper:
    
    def __inint__(self, url, headers):
        self.url = url
        self.headers = headers
        self.news_list = []
        self.params = {}
        self.dom = None
        
    def get_dom(self):
        response = requests.get(self.url, params=self.params, headers=self.headers)
        if response.ok:
            self.dom = html.fromstring(response.text)
        return
    
    def get_list_news(self):
        pprint(self.news_list)

In [3]:
class MailNewsScraper(NewsScraper):
    
    def __init__(self, url, headers):
        super().__inint__(url, headers)
    
    def get_news(self):
        super().get_dom()
        main_news = self.dom.xpath("//div[contains(@class, 'daynews__item')]")
        for news in main_news:
            data_news = {}
            name = news.xpath(".//span[contains(@class, 'photo__title')]/text()")[0]
            name = name.replace(u'\xa0', u' ')
            link = news.xpath(".//a[contains(@class, 'photo')]/@href")[0]
            resp = requests.get(link, params=self.params, headers=self.headers)
            dom_news = html.fromstring(resp.text)
            info = dom_news.xpath("//span[contains(@class, 'note__text')]")
            date = info[0].xpath("./@datetime")[0]
            source = info[1].xpath('..//text()')[1]
            data_news['Заголовок'] = name
            data_news['Ссылка на новость'] = link
            data_news['Дата публикации'] = date
            data_news['Источник'] = source
            self.news_list.append(data_news)
        
        main_news = self.dom.xpath(
            "//ul[@class='list list_type_square list_half js-module']/li[contains(@class, 'list__item')]"
        )
        for news in main_news:
            data_news = {}
            name = news.xpath(".//a[@class='list__text']/text()")[0]
            name = name.replace(u'\xa0', u' ')
            link = news.xpath(".//a[@class='list__text']/@href")[0]
            resp = requests.get(link, params=self.params, headers=self.headers)
            dom_news = html.fromstring(resp.text)
            info = dom_news.xpath("//span[contains(@class, 'note__text')]")
            date = info[0].xpath("./@datetime")[0]
            source = info[1].xpath('..//text()')[1]
            data_news['Заголовок'] = name
            data_news['Ссылка на новость'] = link
            data_news['Дата публикации'] = date
            data_news['Источник'] = source
            self.news_list.append(data_news)
        
    
    

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
}

url = 'https://news.mail.ru'

In [5]:
mail_news = MailNewsScraper(url, headers)
mail_news.get_news()
mail_news.get_list_news()

[{'Дата публикации': '2021-11-24T14:29:29+03:00',
  'Заголовок': 'Зюганов, Кучера и Лоза ответили на письмо врачей антиваксерам',
  'Источник': 'РБК',
  'Ссылка на новость': 'https://news.mail.ru/society/48920567/'},
 {'Дата публикации': '2021-11-24T13:38:10+03:00',
  'Заголовок': 'Фильм Ефремова победил в конкурсе среди осужденных',
  'Источник': 'Фонтанка',
  'Ссылка на новость': 'https://news.mail.ru/society/48922444/'},
 {'Дата публикации': '2021-11-24T11:40:56+03:00',
  'Заголовок': 'Украина начала спецоперацию на границе с Белоруссией',
  'Источник': '© РИА Новости',
  'Ссылка на новость': 'https://news.mail.ru/politics/48918391/'},
 {'Дата публикации': '2021-11-24T12:43:18+03:00',
  'Заголовок': 'В Москве простились с Ниной Руслановой',
  'Источник': '© РИА Новости',
  'Ссылка на новость': 'https://news.mail.ru/society/48920973/'},
 {'Дата публикации': '2021-11-24T13:07:03+03:00',
  'Заголовок': 'В Кремле оценили претензии Бастрыкина к Моргенштерну',
  'Источник': 'Газета.Ру',
 

In [9]:
client = mcl('127.0.0.1', 27017)
db = client['news']
news = db.news
news.create_index('link', unique=True)
def to_mongo(lst):
    for doc in lst:
        try:
            news.insert_one(doc)
        except:
            continue
    return

In [11]:
to_mongo(mail_news.news_list)